# Tool Calling without AI Agent
By the end of this tutorial, you will know how to call functions and store results in the State without invoking an LLM.

## Why it should be useful?
Sometimes, calling an LLM is not as effective as using old-school, self-made functions. If you don't want to lose speed on your invocations, you can use a Custom Agent that mimics a real agent to call a function and store its result in the state.

Key benefits:
*   **Speed:** Direct function calls are orders of magnitude faster (is some cases ~700ms vs ~12,000ms).
*   **Cost:** You don't pay for LLM tokens for deterministic tasks.
*   **Reliability:** You eliminate the chance of the LLM failing to call the function correctly.

In [314]:
from collections.abc import AsyncGenerator
import logging

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.agents import BaseAgent
from google.adk.events import Event, EventActions
from google.adk.agents.invocation_context import InvocationContext


In [315]:
def my_awesome_function(name: str, count: int):
  return {"result": f"Hello, {name}! Made {count} times."}

In [ ]:
class CustomAgent(BaseAgent):
    """Makes a 'my_awesome_function' call"""

    def __init__(self, name: str):
        super().__init__(name=name)

    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        tool_context_instance = ToolContext(ctx)
        tool = FunctionTool(my_awesome_function)
        
        call_args = {
            "name": "Pavel",
            "count": 5,
        }
        logging.info(
            f"[{self.name}] Calling the function"
        )
        # Tools converted with FunctionTool are now having .run_async which is alike runnung a runner
        result = await tool.run_async(
            args=call_args,
            tool_context=tool_context_instance
        )
        # Make a right format for our response of the tool
        function_response_part = types.Part.from_function_response(
            name=tool.name,
            response=result,
        )
        event_content = types.Content(parts=[function_response_part])

        yield Event(
            author = self.name,
            actions=EventActions(state_delta={"TESTING": result}, skip_summarization=True), #To make a response of the Agent as final you can place marker skip_summarization = True
            content=event_content
        )

In [317]:
root_agent = CustomAgent(name="Testing_agent")

In [ ]:
APP_NAME, USER_ID, SESSION_ID = "Test", "User1", "001"
async def setup_session_and_runner():
    session_service = InMemorySessionService()
    session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    logging.info(f"Initial session state: {session.state}")
    runner = Runner(
        agent=root_agent, # Pass the custom agent
        app_name=APP_NAME,
        session_service=session_service
    )
    return session_service, runner

In [319]:
async def call_agent_async(input: str):
    """
    Sends a new topic to the agent (overwriting the initial one if needed)
    and runs the workflow.
    """

    session_service, runner = await setup_session_and_runner()

    current_session = await session_service.get_session(app_name=APP_NAME, 
                                                  user_id=USER_ID, 
                                                  session_id=SESSION_ID)
    if not current_session:
        logging.error("Session not found!")
        return

    content = types.Content(role='user', parts=[types.Part(text=f"{input}")])

    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    async for event in events:
        if event.is_final_response() and event.content and event.content.parts:
            first_part = event.content.parts[0]
            final_output = ""
            
            if first_part.text:
                final_output = first_part.text
            elif first_part.function_response:
                # Этот блок сработает в твоем примере
                response_data = first_part.function_response.response
                final_output = f"Function '{first_part.function_response.name}' returned: {response_data}"
            
            logging.info(f"Final output from [{event.author}]: {final_output}")

    final_session = await session_service.get_session(app_name=APP_NAME, 
                                                user_id=USER_ID, 
                                                session_id=SESSION_ID)
    print("\n--- Final Session State ---")
    import json
    print(json.dumps(final_session.state, indent=2))
    print("-------------------------\n")

In [320]:
text = "Any text"
await call_agent_async(text)


--- Final Session State ---
{
  "TESTING": {
    "result": "Hello, Pavel! Made 5 times."
  }
}
-------------------------

